# Foxglove Data Platform Demo Notebook

This notebook demonstrates retrieving data and events from Foxglove Data Platform.


In [ ]:
# First let's we can create an instance of our client.

from foxglove_data_platform.client import Client, OutputFormat

client = Client(token="fox_sk_CFlhbEgxMQbHBQGRCNE7c3CBaOxtrpk5")

In [ ]:
# Next let's use the client to list our imports.

import pandas as pd

imports = sorted(client.get_imports(), key=lambda i: i["start"])
pd.DataFrame(imports, columns=["device_id", "start", "end", "filename"]).head()

In [ ]:
# This is what a single import looks like

imports[0]

In [ ]:
# Now let's download some GPS data.

gps_data = client.download_data(
    device_id=imports[1]["device_id"],
    start=imports[1]["start"],
    end=imports[1]["end"],
    topics=["/gps"],
    callback=lambda progress: print(".", end=""),
)

In [ ]:
# Let's read some messages from our mcap stream.

from mcap_ros1.ros1_decoder import Ros1Decoder

gps_messages = [
    (message.latitude, message.longitude)
    for topic, record, message in Ros1Decoder(gps_data).messages
]

pd.DataFrame(gps_messages[0:3], columns=["lat", "lon"])

In [ ]:
# We can use our GPS messages to plot our course on the map.

import folium

figure = folium.Figure(width=640, height=480)
map = folium.Map(location=gps_messages[0], zoom_start=200, width="100%")
folium.PolyLine(
    locations=gps_messages,
    weight=10,
    color="orange",
).add_to(map)
map.add_to(figure)

In [ ]:
# We can fetch acceleration data that spans multiple imports

imu_data = client.download_data(
    device_id=imports[0]["device_id"],
    start=imports[0]["start"],
    end=imports[-1]["end"],
    topics=["/imu"],
    callback=lambda progress: print(".", end=""),
)

imu_messages = [
    {
        "time": pd.Timestamp(message.header.stamp.to_nsec(), unit="ns").isoformat(),
        "accel_x": message.linear_acceleration.x,
        "accel_y": message.linear_acceleration.y,
    }
    for topic, record, message in Ros1Decoder(imu_data).messages
]
imu_frame = pd.DataFrame(imu_messages)

In [ ]:
imu_frame.plot(x="time", figsize=(10, 6), rot=45)

In [ ]:
markers_data = client.download_data(
    device_id=imports[1]["device_id"],
    start=imports[1]["start"],
    end=imports[1]["end"],
    topics=["/markers/annotations"],
    callback=lambda progress: print(".", end=""),
)

In [ ]:
import matplotlib as mpl

from pandasql import sqldf

color_to_classname = {
    "#000000": "noise",
    "#468250": "animal",
    "#0000e6": "human.pedestrian.adult",
    "#87ceeb": "human.pedestrian.child",
    "#f08080": "human.pedestrian.construction_worker",
    "#db7093": "human.pedestrian.personal_mobility",
    "#000080": "human.pedestrian.police_officer",
    "#f08080": "human.pedestrian.stroller",
    "#8a2be2": "human.pedestrian.wheelchair",
    "#708090": "movable_object.barrier",
    "#d2691e": "movable_object.debris",
    "#696969": "movable_object.pushable_pullable",
    "#2f4f4f": "movable_object.trafficcone",
    "#bc8f8f": "static_object.bicycle_rack",
    "#dc143c": "vehicle.bicycle",
    "#ff7f50": "vehicle.bus.bendy",
    "#ff4500": "vehicle.bus.rigid",
    "#ff9e00": "vehicle.car",
    "#e99646": "vehicle.construction",
    "#ffd700": "vehicle.emergency.ambulance",
    "#ffd700": "vehicle.emergency.police",
    "#ff3d63": "vehicle.motorcycle",
    "#ff8c00": "vehicle.trailer",
    "#ff6347": "vehicle.truck",
    "#00cfbf": "flat.driveable_surface",
    "#af004b": "flat.other",
    "#4b004b": "flat.sidewalk",
    "#70b43c": "flat.terrain",
    "#deb887": "static.manmade",
    "#ffe4c4": "static.other",
    "#00af00": "static.vegetation",
    "#fff0f5": "vehicle.ego"
}

markers_messages = []
for topic, record, message in Ros1Decoder(markers_data).messages:
  for marker in message.markers:
    color = mpl.colors.to_hex([marker.color.r, marker.color.g, marker.color.b])
    class_name = color_to_classname[color]
    markers_messages.append((marker.text, class_name))
annotations = pd.DataFrame(markers_messages, columns=["annotation_id", "class_name"])

pysqldf = lambda q: sqldf(q, globals())
pysqldf("SELECT class_name,COUNT(*) as count FROM annotations GROUP BY class_name ORDER BY count DESC")